In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import xarray as xr
import numpy as np

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit 
from teslakit.database import Database
from teslakit.io.matlab import ReadMatfile
from teslakit.rbf import RBF_Reconstruction



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('KWAJALEIN')
           

In [3]:
# --------------------------------------
# load data and set parameters

# TCs copula simulated (dataset) and TCs MDA selection
xds_dataset = db.Load_TCs_r2_sim_params()
xds_subset = db.Load_TCs_r2_mda_params()

# r2 mda TCs solved simulations (not solved inside teslakit)
xds_TCs_sims = db.Load_TCs_r2_mda_Simulations()


In [4]:
# --------------------------------------
# load custom RBFs training subset and dataset to interpolate

# TODO: our datasets (db.Load_TCs_r2_sim_params and db.Load_TCs_r2_mda_params) 
# do not match with currently stored solved simulations.

# we are using matching matlab files for KWAJALEIN TCs simulations
# (overriding subset so it matches solved target)
p_test = op.join(p_data, 'tests', 'RBF')
p_subset = op.join(p_test, 'MDA_1000.mat')
p_dataset = op.join(p_test, 'MULTIVARIATE_100000parameters.mat')

# load subset data
subset = ReadMatfile(p_subset)['Subset']  # pmean, vmean, gamma, delta

# load full dataset for interpolation
dm = ReadMatfile(p_dataset)
dataset= np.column_stack(
    (dm['PMEAN1'], dm['VMEAN'], dm['GAMMA'], dm['DELTA'])
)



## Simulated TCs -  Radial Basis Function

In [5]:
# --------------------------------------
# Extract waves data from TCs simulations (this is the RBFs training target)

print(xds_TCs_sims)
print()

# Normalize data
d_maxis = {}
d_minis = {}
xds_n = xds_TCs_sims.copy()
for k in ['hs', 'tp', 'ss', 'twl']:
    v = xds_n[k].values[:]
    mx = np.max(v)
    mn = np.min(v)
    xds_n[k] =(('storm',), (v-mn)/(mx-mn))
    
    # store maxs and mins for denormalization
    d_maxis[k] = mx
    d_minis[k] = mn

xds_n['dir'] = xds_n['dir'] * np.pi/180
    
print(xds_n)
print()


# Build RBF target numpy array
target = np.column_stack(
    (xds_n['hs'], xds_n['tp'], xds_n['ss'], xds_n['twl'], xds_n['dir'], xds_n['mu'])
)


<xarray.Dataset>
Dimensions:  (storm: 1000)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    hs       (storm) float64 0.13 0.026 4.398 1.352 ... 1.782 8.078 5.11 6.136
    tp       (storm) float64 3.61 1.506 14.71 9.259 ... 6.494 13.16 11.9 13.33
    dir      (storm) float64 42.4 45.0 124.2 267.3 13.2 ... 35.2 23.5 90.0 122.3
    ss       (storm) float64 0.005154 0.007572 0.09592 ... 0.1987 0.07206 0.1122
    twl      (storm) float64 0.07414 0.02044 1.73 0.5898 ... 2.181 1.498 1.863
    mu       (storm) float64 0.7074 0.8419 0.805 0.9491 ... 0.623 0.5043 0.6556

<xarray.Dataset>
Dimensions:  (storm: 1000)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    hs       (storm) float64 0.009134 0.001473 0.3235 ... 0.5946 0.376 0.4515
    tp       (storm) float64 0.1312 0.0 0.823 0.4834 ... 0.7265 0.6484 0.7375
    dir      (storm) float64 0.74 0.7854 2.168 4.665 ...

In [6]:
# --------------------------------------
# RBF Interpolation

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]        # scalar (pmean,  vmean)
ix_directional_subset = [2,3]   # directional (delta, gamma)

# target - scalar / directional indexes
ix_scalar_target = [0,1,2,3,5]  # scalar (Hs, Tp, SS, TWL, MU)
ix_directional_target = [4]     # directional (Dir)

output = RBF_Reconstruction(
    subset, ix_scalar_subset, ix_directional_subset,
    target, ix_scalar_target, ix_directional_target,
    dataset)


ix_scalar: 0,  optimization: 587.50 | interpolation: 72.29
ix_scalar: 1,  optimization: 483.39 | interpolation: 43.54
ix_scalar: 2,  optimization: 543.65 | interpolation: 49.50
ix_scalar: 3,  optimization: 534.73 | interpolation: 40.50
ix_scalar: 5,  optimization: 366.69 | interpolation: 34.23
ix_directional: 4,  optimization: 1064.18 | interpolation: 72.36


In [7]:
# --------------------------------------
# output de-normalization

hs_n = output[:,0]
tp_n = output[:,1]
ss_n = output[:,2]
twl_n = output[:,3]
dir_n = output[:,4]
mu = output[:,5]

# store data as xarray.Dataset
xds_out = xr.Dataset(
    {
        'hs':(('storm',), hs_n * (d_maxis['hs']-d_minis['hs']) + d_minis['hs'] ),
        'tp':(('storm',), tp_n * (d_maxis['tp']-d_minis['tp']) + d_minis['tp'] ),
        'ss':(('storm',), ss_n * (d_maxis['ss']-d_minis['ss']) + d_minis['ss'] ),
        'twl':(('storm',), twl_n * (d_maxis['twl']-d_minis['twl']) + d_minis['twl'] ),
        'dir':(('storm',), dir_n * 180 / np.pi),
        'mu':(('storm',), mu),
    },
    coords = {'storm': np.arange(output.shape[0])}
)
print(xds_out)

db.Save_TCs_sim_r2_rbf_output(xds_out)


<xarray.Dataset>
Dimensions:  (storm: 100000)
Coordinates:
  * storm    (storm) int64 0 1 2 3 4 5 6 ... 99994 99995 99996 99997 99998 99999
Data variables:
    hs       (storm) float64 6.488 2.125 2.428 2.505 ... 5.623 2.632 1.791 3.593
    tp       (storm) float64 12.24 10.92 7.623 10.04 ... 12.79 7.024 8.28 9.868
    ss       (storm) float64 0.1163 0.0632 0.018 ... 0.1009 0.04997 0.1177
    twl      (storm) float64 1.671 0.8837 0.6813 0.82 ... 0.6608 0.6022 1.141
    dir      (storm) float64 95.05 82.92 66.62 111.1 ... 114.1 61.17 88.9 98.49
    mu       (storm) float64 0.6176 0.6227 0.8581 0.725 ... 0.7585 0.7668 0.4756
